# What Features Determine a Borrower's Loan Rate and Prosper Rating
## by Alf Maglalang

## Investigation Overview

> The goal of the investigation is to examine through visualizations which features have relationships with Prosper Rating and Borrower Rate. I am conflating Prosper Rating to mean the features CreditGrade and ProsperRating (numeric) which are mutually exclusive with CreditGrade before mid 2009 and ProsperRating after mid-2009.

> I tracked 4 features (namely `CreditGrade`, ProsperScore, ProsperRating (numeric), ,to see their relationship

> Initially, I thought that I should track Prosper Score. However, upon analysing I noticed that there might be some clerical errors in the column Prosper Score.

> Upon checking for duplicate ListingKey number, I noticed that the duplicates were only duplicates if I dropped the ProsperScore. This was not serendipity. I took a ListingKey that had 6 entries. I visually scanned and noted differences in ProsperScore. I dropped that column first and then checked again for duplicates on all features. And those entries were duplicates with ProsperScore dropped.

> I was not really sure what consequences the duplicates would have in visualizations. I created a new column called ProsperScoreRange and placed the series of "incorrect" ProsperScores as a string of numbers. I then dropped the duplicates ListingKey save one.

> Using the

## Dataset Overview

> This loan data is from a lending company called **Prosper**. It was last updated 3/11/2014. It has 113937 records with 81 features, namely

> `ListingKey`, `ListingNumber`, `ListingCreationDate`, `CreditGrade`, `Term`, `LoanStatus`, `ClosedDate`, `BorrowerAPR`, `BorrowerRate`, `LenderYield`, `EstimatedEffectiveYield`, `EstimatedLoss`, `EstimatedReturn`, `ProsperRating (numeric)`, `ProsperRating (Alpha)`, `ProsperScore`, `ListingCategory (numeric)`, `BorrowerState`, `Occupation`, `EmploymentStatus`, `EmploymentStatusDuration`, `IsBorrowerHomeowner`, `CurrentlyInGroup`, `GroupKey`, `DateCreditPulled`, `CreditScoreRangeLower`, `CreditScoreRangeUpper`, `FirstRecordedCreditLine`, `CurrentCreditLines`, `OpenCreditLines`, `TotalCreditLinespast7years`, `OpenRevolvingAccounts`, `OpenRevolvingMonthlyPayment`, `InquiriesLast6Months`, `TotalInquiries`, `CurrentDelinquencies`, `AmountDelinquent`, `DelinquenciesLast7Years`, `PublicRecordsLast10Years`, `PublicRecordsLast12Months`, `RevolvingCreditBalance`, `BankcardUtilization`, `AvailableBankcardCredit`, `TotalTrades`, `TradesNeverDelinquent (percentage)`, `TradesOpenedLast6Months`, `DebtToIncomeRatio`, `IncomeRange`, `IncomeVerifiable`, `StatedMonthlyIncome`, `LoanKey`, `TotalProsperLoans`, `TotalProsperPaymentsBilled`, `OnTimeProsperPayments`, `ProsperPaymentsLessThanOneMonthLate`, `ProsperPaymentsOneMonthPlusLate`, `ProsperPrincipalBorrowed`, `ProsperPrincipalOutstanding`, `ScorexChangeAtTimeOfListing`, `LoanCurrentDaysDelinquent`, `LoanFirstDefaultedCycleNumber`, `LoanMonthsSinceOrigination`, `LoanNumber`, `LoanOriginalAmount`, `LoanOriginationDate`, `LoanOriginationQuarter`, `MemberKey`, `MonthlyLoanPayment`, `LP_CustomerPayments`, `LP_CustomerPrincipalPayments`, `LP_InterestandFees`, `LP_ServiceFees`, `LP_CollectionFees`, `LP_GrossPrincipalLoss`, `LP_NetPrincipalLoss`, `LP_NonPrincipalRecoverypayments`, `PercentFunded`, `Recommendations`, `InvestmentFromFriendsCount`, `InvestmentFromFriendsAmount`, `Investors`

> Please click on the following link to see the definitions of the data's features. [Prosper Data Dictionary to Explain Dataset's Variables](https://www.google.com/url?q=https://docs.google.com/spreadsheet/ccc?key%3D0AllIqIyvWZdadDd5NTlqZ1pBMHlsUjdrOTZHaVBuSlE%26usp%3Dsharing&sa=D&ust=1554486256024000)

> For my analysis, I will look at these features:
> `ListingKey`, `EmploymentStatusDuration`, `EmploymentStatus`, `LoanOriginalAmount`, `CreditGrade`, `ProsperRating (numeric)`, `ProsperScore`, `BorrowerRate`, `CreditScoreRangeUpper`, `CurrentDelinquencies`, `IsBorrowerHomeowner`, `CurrentCreditLines`, `AvailableBankcardCredit`

> I created 3 new features.

> - `ProsperScoreRange` -- a column containing a string of "incorrect" `ProsperScore (numeric)` of duplicated `ListingKey` IDs
> - `ProsperRatingGrade` -- a column combining `CreditGrade` and `ProsperScore (numeric)`
> - `post2009` -- a column that is a form of boolean mask differentiating if a `ProsperRatingGrade` is originally from `CreditGrade` or from `ProsperRating (numeric)`.

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

In [ ]:
# load in the dataset into a pandas dataframe


> Note that the above cells have been set as "Skip"-type slides. That means
that when the notebook is rendered as http slides, those cells won't show up.

## (Visualization 1)

> Write a comment about your visualization here. The visualization should be in
the next cell, as a sub-slide type. Make sure your visualizations are polished!

## (Visualization 2)

> You should have at least three visualizations in your presentation,
but feel free to add more if you'd like!

## (Visualization 3)



> Once you're ready to finish your presentation, check your output by using
nbconvert to export the notebook and set up a server for the slides. From the
terminal or command line, use the following expression:
> > `jupyter nbconvert <file_name>.ipynb --to slides --post serve --template output_toggle`

> This should open a tab in your web browser where you can scroll through your
presentation. Sub-slides can be accessed by pressing 'down' when viewing its parent
slide. Make sure you remove all of the quote-formatted guide notes like this one
before you finish your presentation!